In [ ]:
import requests
import pandas as pd
import os
import re

url = "https://cfas.apprentissage.beta.gouv.fr/api/v1/organisation/organismes"

#Anonymisation des cookies car données personnelles (besoin d'un compte pour se connecter au site et donc l'API)
cookies = {
    "": ""
}

rep = requests.get(url, cookies=cookies)

if rep.status_code != 200:
    raise Exception(f"Erreur lors de la récupération des organismes : {rep.status_code}")

data = pd.DataFrame(rep.json())

if "_id" not in data.columns or "raison_sociale" not in data.columns:
    raise ValueError("Les colonnes '_id' et 'raison_sociale' sont absentes des données récupérées.")

id_to_name = dict(zip(data["_id"], data["raison_sociale"]))

output_dir = "organismes_csv"
os.makedirs(output_dir, exist_ok=True)

for _id, raison_sociale in id_to_name.items():
    api_url = f"https://cfas.apprentissage.beta.gouv.fr/api/v1/organismes/{_id}/indicateurs/effectifs/par-formation?date=2025-03-03T09:38:10.526Z"
    
    try:
        rep = requests.get(api_url, cookies=cookies)
        
        if rep.status_code == 200:
            data_api = pd.DataFrame(rep.json())

            if not data_api.empty:
                raison_propre = re.sub(r'[\\/*?:"<>|]', '', raison_sociale)

                fichier_path = os.path.join(output_dir, f"{raison_propre}.csv")
                data_api.to_csv(fichier_path, index=False)
                print(f"Fichier créé : {fichier_path}")
            else:
                print(f"Données vides pour {_id}, fichier non créé.")

        else:
            print(f"Erreur {rep.status_code} pour l'ID {_id}")

    except Exception as e:
        print(f"Erreur lors du traitement de l'ID {_id} : {e}")


In [ ]:
dossier = "organismes_csv"

df_list = []


for fichier in os.listdir(dossier):
    if fichier.endswith(".csv"): 
        _id = os.path.splitext(fichier)[0] 
        
        df = pd.read_csv(os.path.join(dossier, fichier))
        

        df["_id"] = _id

        df_list.append(df)

df_final = pd.concat(df_list, ignore_index=True)

df_final.to_csv("grand_fichier.csv", index=False)

print("Le fichier 'grand_fichier.csv' a été créé.")


Le fichier 'grand_fichier.csv' a été créé.


In [33]:
import pandas as pd

grand_fichier = pd.read_csv("grand_fichier.csv")

colonnes_a_garder = ["_id", "uai", "siret", "reseaux", "enseigne", "adresse"]
data = data[colonnes_a_garder]

df_final = grand_fichier.merge(data, on="_id", how="left")

autres_colonnes = [col for col in df_final.columns if col not in colonnes_a_garder]
colonnes_ordre = colonnes_a_garder + autres_colonnes
df_final = df_final[colonnes_ordre]

df_final.to_csv("grand_fichier_complet.csv", index=False)


In [ ]:
url = "https://cfas.apprentissage.beta.gouv.fr/api/v1/organisation/organismes"

#Anonymisation des cookies car données personnelles (besoin d'un compte pour se connecter au site et donc l'API)
cookies = {
    "": ""
}

rep = requests.get(url, cookies=cookies)

if rep.status_code != 200:
    raise Exception(f"Erreur lors de la récupération des organismes : {rep.status_code}")

data = pd.DataFrame(rep.json())
data.to_csv("organismes_total.csv", index=False)